In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.4516],
        [0.5172]], grad_fn=<AddmmBackward0>)

## 参数访问

In [3]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0205, -0.1595,  0.2746,  0.1488, -0.0511,  0.3292, -0.1727, -0.1077]])), ('bias', tensor([0.3476]))])


In [4]:
print(net[0].state_dict())

OrderedDict([('weight', tensor([[ 0.4450, -0.2869, -0.1569, -0.4670],
        [-0.4799, -0.4326, -0.4883, -0.3525],
        [ 0.4793,  0.2976,  0.4723, -0.3660],
        [-0.4344, -0.4783, -0.0603,  0.4514],
        [ 0.2143,  0.4647, -0.3964, -0.4162],
        [ 0.4525,  0.3947,  0.4410, -0.2400],
        [-0.1721, -0.4625, -0.3737, -0.3290],
        [-0.2019,  0.2819,  0.4831, -0.3437]])), ('bias', tensor([-0.4420,  0.2074, -0.1590,  0.4899, -0.1096, -0.2476,  0.2828, -0.2150]))])


In [5]:
print(net[1].state_dict()) # relu层哪来的权重，和偏差 hhh；

OrderedDict()


### 目标参数

In [6]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.3476], requires_grad=True)
tensor([0.3476])


In [7]:
print(net[2].weight)

Parameter containing:
tensor([[-0.0205, -0.1595,  0.2746,  0.1488, -0.0511,  0.3292, -0.1727, -0.1077]],
       requires_grad=True)


In [9]:
print(net[0].weight)

Parameter containing:
tensor([[ 0.4450, -0.2869, -0.1569, -0.4670],
        [-0.4799, -0.4326, -0.4883, -0.3525],
        [ 0.4793,  0.2976,  0.4723, -0.3660],
        [-0.4344, -0.4783, -0.0603,  0.4514],
        [ 0.2143,  0.4647, -0.3964, -0.4162],
        [ 0.4525,  0.3947,  0.4410, -0.2400],
        [-0.1721, -0.4625, -0.3737, -0.3290],
        [-0.2019,  0.2819,  0.4831, -0.3437]], requires_grad=True)


In [10]:
net[2].weight.grad == None

True

###  一次性访问所有参数

In [13]:
# Using * in front of a list expands out the members as individual arguments.
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [12]:
print([(name, param.shape) for name, param in net[0].named_parameters()])
print([(name, param.shape) for name, param in net.named_parameters()])

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
[('0.weight', torch.Size([8, 4])), ('0.bias', torch.Size([8])), ('2.weight', torch.Size([1, 8])), ('2.bias', torch.Size([1]))]


### 从嵌套块收集参数

In [14]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套 4个block1
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3829],
        [-0.3829]], grad_fn=<AddmmBackward0>)

In [15]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [16]:
rgnet[0][1]

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
)

In [17]:
rgnet[0][1][0].bias.data

tensor([ 0.2428,  0.0356,  0.1816, -0.1282,  0.0024, -0.4137, -0.2773, -0.2174])

## 参数初始化

### 内置初始化

In [18]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [19]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0096, -0.0068,  0.0140, -0.0010]), tensor(0.))

In [20]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [21]:
net[2].weight.data[0], net[2].bias.data[0]

(tensor([1., 1., 1., 1., 1., 1., 1., 1.]), tensor(0.))

我们还可以对某些块应用不同的初始化方法。 

In [22]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.6192, -0.5023, -0.3234, -0.4874])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化

深度学习框架没有提供我们需要的初始化方法。 在下面的例子中，我们使用以下的分布为任意权重参数w定义初始化方法：

$$\begin{split}\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}\end{split}$$

In [23]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  0.0000, -0.0000,  0.0000],
        [ 6.6855,  0.0000, -6.2908,  0.0000]], grad_fn=<SliceBackward0>)

In [24]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

##  参数绑定

In [25]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


这个例子表明第三个和第五个神经网络层的参数是绑定的。 它们不仅值相等，而且由相同的张量表示。 因此，如果我们改变其中一个参数，另一个参数也会改变。 你可能会思考：当参数绑定时，梯度会发生什么情况？ 答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层 （即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。